In [3]:
import re
from bs4 import BeautifulSoup
import urllib
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import time
# from webdriver_manager.chrome import ChromeDriverManager
def Control(data):
    #data = BeautifulSoup(webpage_content, 'html.parser')
    temp = data.find_all("p",{"class":"ci-html-content"})
    for i in temp:
        if i.has_attr('class'):
            if 'ci-html-content' in i['class'] and len(i['class']) == 1:
                j = i.parent
                j = j.parent
                j = j.parent
                j = j.parent
                j = j.parent
                k = j.find("span").text
    b = float(k)
    return b

def check_innings(d):
    global innings_num
    if game_format.lower()=="t20i" or game_format.lower()=="odi":
        innings_num=2
        p_source=d.page_source
        p_source=BeautifulSoup(p_source, 'lxml')
        e=d.find_element_by_xpath('''//*[@id="main-container"]/div[5]/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[2]/div/div/span''')
        if e.text.lower()!=country:
            webdriver.ActionChains(d).move_to_element(e).click(e).perform()
            x=d.find_element_by_xpath('''//*[@id="tippy-10"]/div/div/div/div/div/ul/li[1]''')
            if x.text.lower()==country:
                webdriver.ActionChains(d).move_to_element(x).click(x).perform()
                innings_num=1
    else:
        p_source=d.page_source
        p_source=BeautifulSoup(p_source, 'lxml')
        e=d.find_element_by_xpath('''//*[@id="main-container"]/div[5]/div[1]/div/div[3]/div[1]/div[2]/div[1]/div[1]/div/div[2]/div/div/span''')
        webdriver.ActionChains(d).move_to_element(e).click(e).perform()
        x=d.find_elements_by_xpath('''//*[@id="tippy-10"]/div/div/div/div/div/ul/li''')
        for i in x:
            if i.text.split(' ')[0].lower()==country and int(i.text.split(' ')[1][0])==innings_num+1:
                ele=i
                break
        innings_num=innings_num+1
        webdriver.ActionChains(d).move_to_element(ele).click(ele).perform()

def scroll(driver):
    a = 0
    b = 0.1
    control = "start"
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "tbl-feed-header-text")))
    for i in range(100):
        if control != "end":
            webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
            page_source=driver.page_source
            data = BeautifulSoup(page_source, 'lxml')
            a = Control(data)
            i = i+1
        if  b == a:
            control = "end"
    
def Test():
    options = Options()
    options.headless = True
    options.add_argument("--window-size=1920,1200")
    driver=webdriver.Chrome("C://Users/saita/Downloads/chromedriver_win32/chromedriver.exe",options=options)
    driver.get(l)
    driver.maximize_window()
    temp=[]
    for i in range(2):
        check_innings(driver)
        scroll(driver)
        temp.append(scrape(driver))
        print("Done1")
    df1=pd.DataFrame(temp[0])
    df2=pd.DataFrame(temp[1])
    df=pd.concat([df1, df2], axis=0)
    return df
    
def start(link,format1,team):
    global innings_num
    innings_num=0
    global game_format
    global country
    global l
    l=link
    country=team
    game_format=format1
    sol=None
    if game_format.lower()=='test':
        sol=Test()
    else:
        options = Options()
        options.headless = True
        options.add_argument("--window-size=1920,1200")
        driver=webdriver.Chrome("C://Users/saita/Downloads/chromedriver_win32/chromedriver.exe",options=options)
        driver.get(link)
        driver.maximize_window()
        check_innings(driver)
        scroll(driver)
        sol=scrape(driver)
    return sol
    
def scrape(d):
    data = d.page_source
    data = BeautifulSoup(data, 'lxml')
    commentary = []
    runs = []
    innings=[]
    overs=[]
    ball_Number=[]
    striker=[]
    non_striker=[]
    bowler=[]
    extra=[]
    batter_Runs=[]
    extra_runs=[]
    runs_for_ball=[]
    boundary=[]
    is_wicket=[]
    player_out=[]
    Type_of_wicket=[]
    Fielders_involved=[]
    lengths=['shortish','flat','short of','short','back of','full','good','length','slot','yorker','toss','volley','bounce']
    ball_length=[]
    temp = data.find_all("p",{"class":"ci-html-content"})
    for i in temp:
        if i.has_attr('class'):
            if 'ci-html-content' in i['class'] and len(i['class']) == 1:
                commentary.append(i.text)
                j = i.parent
                b = j.previous_sibling
                sp1 = b.find("span").text
                data=sp1.split(',')[0].split('to')
                bowler.append(data[0])
                striker.append(data[1])
                j = j.parent
                j = j.parent
                r=j
                j = j.parent
                j = j.parent
                k = j.find("span").text
                ch=j.findChildren("span")
                overs.append(ch[0].text)
                if ch[1].text!='W':
                    runs.append(ch[1].text)
                else:
                    r=r.find_next_sibling('div')
                    r=r.find("strong").text
                    if 'st' in r:
                        fielder=r.split('st')[1].split(' b ')[0]
                        runs.append("W Stump_Out-"+fielder)
                    elif 'lbw' in r:
                        runs.append("W Leg_Before_Wicket-")
                    elif 'c & b' in r:
                        fielder=r.split('c & b')[1].split(' ')[0]
                        runs.append("W Caught_and_Bowled-"+fielder)
                    elif 'hit wicket' in r:
                        runs.append("W Hit_Wicket-")
                    elif ' c ' in r:
                        fielder=r.split(' c ')[1].split(' b ')[0]
                        runs.append("W Caught-"+fielder)
                    elif 'run out' in r:
                        p_out=r.split('run out')[0]
                        fielder=r.split('run out')[1].split(') ')[0]
                        fielder=fielder.replace('(','')
                        if '/' in fielder:
                            fielder=fielder.split('/')[0]+","+fielder.split('/')[1]
                        runs.append("W Run_Out-"+fielder+"$"+p_out)
                    elif 'b' in r:
                        runs.append("W Bowled-"+'')
    commentary.reverse()
    bowler.reverse()
    striker.reverse()
    runs.reverse()
    overs.reverse()
    #finding suitable words from the commentary
    find_words = re.compile('|'.join(lengths)).findall
    for i,s in enumerate(commentary):
        words = find_words(s.lower())
        if words:
            ball_length.append(words)
        else:
            ball_length.append(None)
    #Filling lists with default values
    extra_runs=[0]*len(runs)
    extra=[None]*len(runs)
    is_wicket=[0]*len(runs)
    Type_of_wicket=[None]*len(runs)
    Fielders_involved=[None]*len(runs)
    boundary=[0]*len(runs)
    player_out=[None]*len(runs)
    batter_Runs=[0]*len(runs)
    runs_for_ball=[0]*len(runs)

    innings=[innings_num]*len(runs)
    ### Finding Ball_lengths
    for i,j in enumerate(ball_length):
        if j == None:
            continue
        if 'toss' in j:
            ball_length[i]="Full Toss"
        elif 'yorker' in j:
            ball_length[i]="Yorker"
        elif 'back of' in j:
            ball_length[i]="Back of Length"
        elif 'short of' in j and 'good' in j:
            ball_length[i]="Back of Length"
        elif 'full' in j or 'flat' in j:
            ball_length[i]="Full Length"
        elif 'slot' in j or 'volley' in j:
            ball_length[i]='Slot'
        elif 'short' in j or 'short of' in j or 'shortish' in j:
            ball_length[i]="Short Ball"
        elif 'bounce' in j:
            ball_length[i]="Bouncer"
        elif 'good' in j or 'length' in j:
            ball_length[i]="Good Length"
            
    ### Assigning values to all dependent features like wicket type, extras, etc from the extracted data
    for i,j in enumerate(runs):
        if type(j)==type('s'):
            if len(j)==1 and j!='•':
                runs[i]=int(j)
        if j=='•':
            runs[i]=0
        elif 'W' in str(j):
            is_wicket[i]=1
            Type_of_wicket[i]=j.split(" ")[1].split('-')[0]
            if Type_of_wicket[i]=="Run_Out":
                player_out[i]=j.split('$')[1]
            else:
                player_out[i]=striker[i]
            temp=j.split('-')
            if temp[0].split(' ')[1] not in ['Bowled','Leg_Before_Wicket','Caught_and_Bowled','Hit_Wicket']:
                Fielders_involved[i]=temp[1]
        elif 'w' in str(j):
            extra[i]="Wide"
            extra_runs[i]=j[0]
            batter_Runs[i]=0
            runs_for_ball[i]=extra_runs[i]
        elif 'nb' in str(j):
            extra[i]="No_Ball"
            extra_runs[i]=1
            batter_Runs[i]=int(j[0])-1
            if batter_Runs[i]==4:
                boundary[i]='Four'
            if batter_Runs[i]==6:
                boundary[i]='Sixer'
            runs_for_ball[i]=int(j[0])
        elif 'lb' in str(j):
            extra[i]="Leg Byes"
            extra_runs[i]=int(j[0])
            batter_Runs[i]=0
            runs_for_ball[i]=extra_runs[i]
        elif 'b' in str(j):
            extra[i]="Byes"
            extra_runs[i]=int(j[0])
            batter_Runs[i]=0
            runs_for_ball[i]=extra_runs[i]
        else:
            runs_for_ball[i]=j
            batter_Runs[i]=j
            extra_runs[i]=0
            if runs[i]==4:
                boundary[i]='Four'
            if runs[i]==6:
                boundary[i]='Sixer'
    ball_Number=[i for i in range(1,(len(overs)+1))]

    data = {"overs":overs,"Ball_Number":ball_Number,"innings":innings,"Striker":striker,"Bowler":bowler,"Extra":extra,"Batter_Runs":batter_Runs,"Extra_runs":extra_runs,"Runs_for_ball":runs_for_ball,"Boundary":boundary,"is_wicket":is_wicket,"player_out":player_out,"Type_of_wicket":Type_of_wicket,"Fielder_involved":Fielders_involved,"length":ball_length}
    print("Done")
    return data

In [4]:
# d=start("https://www.espncricinfo.com/series/icc-world-test-championship-2019-2021-1195334/india-vs-new-zealand-final-1249875/ball-by-ball-commentary","test","ind")
data=start("https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/india-vs-pakistan-16th-match-group-2-1298150/ball-by-ball-commentary","t20i","ind")
# data=scrape(d)","t20i","ind")

Done


In [26]:
import pandas as pd
df=pd.DataFrame(data)

In [27]:
for i in df.columns:
    if df[i].dtype==df['Striker'].dtype:
        df[i]=df[i].str.strip()

In [35]:
values={'Extra_runs':0,'Batter_Runs':0,'Boundary':0,'Runs_for_ball':0}
df.fillna(value=values,inplace=True)

In [36]:
df

,overs,Ball_Number,innings,Striker,Bowler,Extra,Batter_Runs,Extra_runs,Runs_for_ball,Boundary,is_wicket,player_out,Type_of_wicket,Fielder_involved,length
0,0.1,1,2,Rahul,Shaheen Shah Afridi,None,0,0,0,0,0,None,None,None,Full Toss
1,0.2,2,2,Rahul,Shaheen Shah Afridi,None,1,0,1,0,0,None,None,None,Full Length
2,0.3,3,2,Rohit Sharma,Shaheen Shah Afridi,None,1,0,1,0,0,None,None,None,Full Toss
3,0.4,4,2,Rahul,Shaheen Shah Afridi,None,1,0,1,0,0,None,None,None,Yorker
4,0.5,5,2,Rohit Sharma,Shaheen Shah Afridi,None,0,0,0,0,0,None,None,None,Full Length
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
120,19.4,121,2,Kohli,Nawaz,Wide,0,1,1,0,0,None,None,None,Yorker
121,19.4,122,2,Kohli,Nawaz,Byes,0,0,0,0,0,None,None,None,Full Length
122,19.5,123,2,Karthik,Nawaz,None,0,0,0,0,1,Karthik,Stump_Out,†Mohammad Rizwan,Full Length
123,19.6,124,2,Ashwin,Nawaz,Wide,0,1,1,0,0,None,None,None,Full Length
